##### imports and setup

In [ ]:
import tensorflow as tf
import pandas as pd

## data set

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

CSV_COLUMN_NAMES = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"]
SPECIES = ["Setosa", "Versicolor", "Virginica"]

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
print(train.head())

train_y = train.pop("Species")
test_y = test.pop("Species")
print(train.head())
print(train_y.head())

print(train.shape)

## input function

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

## feature columns

In [ ]:
# Feature columns describe how to use the input
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


## build model with dnn (deep neural network) & train

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes
    hidden_units=[30, 10],
    # 3 possible classifications
    n_classes=3)

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),  # include a lambda to avoid creating an inner function previously in input_fn
    steps=5000)  # steps: max number of steps (doing gradient descent)

## Evaluation

In [ ]:
# note: not specify steps due to evaluation looks the data only 1 time
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print("\nTest set accuracy: {accuracy:0.3f}\n".format(**eval_result))

## predictions

In [ ]:
# user input based
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict["class_ids"][0]
    probability = pred_dict["probabilities"][class_id]

    print("Prediction is '{}' ({:.1f}%)".format(
        SPECIES[class_id], 100 * probability))

In [ ]:
# Generate predictions from the model
expected = ["Setosa", "Versicolor", "Virginica"]
predict_x = {
    "SepalLength": [5.1, 5.9, 6.9],
    "SepalWidth": [3.3, 3.0, 3.1],
    "PetalLength": [1.7, 4.2, 5.4],
    "PetalWidth": [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict["class_ids"][0]
    probability = pred_dict["probabilities"][class_id]

    print("Prediction is '{}' ({:.1f}%), expected '{}'".format(
        SPECIES[class_id], 100 * probability, expec))